In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from utilities import train_model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 30
BATCH_SIZE = 16

TRAIN_TEST_SPLIT = 0.78
MIN = 1
TARGET = 'target.csv'
FILE_NAME = '0.csv'
FILE_NAME1 = '1.csv'
FILE_NAME2 = '2.csv'
FILE_NAME3 = '3.csv'


def readfile(file_name):
    file = open(file_name, 'r', encoding='utf-8')
    next(file)
    data = []
    for line in (file):
        values = line.split(',')
        data.append(float(values[0]))
    file.close()
    return np.array(data, dtype=np.float32)

# Read data and split up into train and test data.
sales = readfile(TARGET)
zero = readfile(FILE_NAME)
um = readfile(FILE_NAME1)
dois = readfile(FILE_NAME2)
tres = readfile(FILE_NAME3)


months = len(sales)
split = int(months * TRAIN_TEST_SPLIT)

train_sales = sales[0:split]
test_sales = sales[split:]

train_sales0 = zero[0:split]
test_sales0 = zero[split:]

train_sales1 = um[0:split]
test_sales1 = um[split:]

train_sales2 = dois[0:split]
test_sales2 = dois[split:]

train_sales3 = tres[0:split]
test_sales3 = tres[split:]

In [ ]:
test_sales

In [ ]:
# Plot dataset
x = range(len(sales))
plt.plot(x, sales, 'r-', label='car sales')
plt.title('Sales')
#plt.axis('')
plt.xlabel('Safra')
plt.ylabel('Target')
plt.legend()
plt.show()

In [ ]:
print(test_sales[MIN:])
print(test_sales[MIN-1:-1])

In [ ]:
# Plot naive prediction
test_output = test_sales[MIN:]
naive_prediction = test_sales[MIN-1:-1]
x = range(len(test_output))
plt.plot(x, test_output, 'g-', label='Valores reais')
plt.plot(x, naive_prediction, 'm-', label='Previsão ingênua')
plt.title('Venda de carros')
#plt.axis([0, len(test_output), 9500, 13000.0])
plt.xlabel('Safra')
plt.ylabel('Venda de carros mensal')
plt.legend()
plt.show()

In [ ]:
test_sales[MIN:]

In [ ]:
# Standardize train and test data.
# Use only training seasons to compute mean and stddev.
mean = np.mean(train_sales)
stddev = np.std(train_sales)
train_sales_std = (train_sales - mean)/stddev
test_sales_std = (test_sales - mean)/stddev

mean0 = np.mean(train_sales0)
stddev0 = np.std(train_sales0)
train_sales_std0 = (train_sales0 - mean0)/stddev0
test_sales_std0 = (test_sales0 - mean0)/stddev0

mean1 = np.mean(train_sales1)
stddev1 = np.std(train_sales1)
train_sales_std1 = (train_sales1 - mean1)/stddev1
test_sales_std1 = (test_sales1 - mean1)/stddev1

mean2 = np.mean(train_sales2)
stddev2 = np.std(train_sales2)
train_sales_std2 = (train_sales2 - mean2)/stddev2
test_sales_std2 = (test_sales2 - mean2)/stddev2

mean3 = np.mean(train_sales3)
stddev3 = np.std(train_sales3)
train_sales_std3 = (train_sales3 - mean3)/stddev3
test_sales_std3 = (test_sales3 - mean3)/stddev3

In [ ]:
# Create train examples.valores
train_months = len(train_sales)
train_X = np.zeros((train_months-MIN, train_months-1, 4), dtype=np.float32)
train_y = np.zeros((train_months-MIN, 1), dtype=np.float32)
for i in range(0, train_months-MIN):
    train_X[i, -(i+MIN):, 0] = train_sales_std[0:i+MIN]
    train_X[i, -(i+MIN):, 1] = train_sales_std0[0:i+MIN]
    train_X[i, -(i+MIN):, 2] = train_sales_std1[0:i+MIN]
    train_X[i, -(i+MIN):, 3] = train_sales_std2[0:i+MIN]
    #train_X[i, -(i+MIN):, 4] = train_sales_std3[0:i+MIN]
    train_y[i, 0] = train_sales_std[i+MIN]

# Create test examples.
test_months = len(test_sales)
test_X = np.zeros((test_months-MIN, test_months-1, 4), dtype=np.float32)
test_y = np.zeros((test_months-MIN, 1), dtype=np.float32)
for i in range(0, test_months-MIN):
    test_X[i, -(i+MIN):, 0] = test_sales_std[0:i+MIN]
    test_X[i, -(i+MIN):, 1] = test_sales_std0[0:i+MIN]
    test_X[i, -(i+MIN):, 2] = test_sales_std1[0:i+MIN]
    test_X[i, -(i+MIN):, 3] = test_sales_std2[0:i+MIN]
    #test_X[i, -(i+MIN):, 4] = test_sales_std3[0:i+MIN]
    test_y[i, 0] = test_sales_std[i+MIN]


# Create Dataset objects.
trainset = TensorDataset(torch.from_numpy(train_X).clone(), torch.from_numpy(train_y))
testset = TensorDataset(torch.from_numpy(test_X).clone(), torch.from_numpy(test_y))

In [ ]:
# Custom layer that retrieves only last time step from RNN output.
class LastTimestep(nn.Module):
    def forward(self, inputs):
        return inputs[1][0]

# Create RNN model
model = nn.Sequential(
    nn.RNN(4, 128, nonlinearity='relu', batch_first=True),
    LastTimestep(),
    nn.Linear(128, 1)
)

# Loss function and optimizer.
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.MSELoss()

# Train model.
train_model(model, device, EPOCHS, BATCH_SIZE, trainset, testset,
            optimizer, loss_function, 'mae')

In [ ]:
mae = [0.2349,0.0134,0.0026,0.0015,0.0014,0.0014,0.0010,0.0010,0.0009,0.0008,0.0008,0.0010,0.0010,0.0007,0.0007,0.0009,0.0007,0.0006,0.0006,0.0005,0.0005,0.0006,0.0006,0.0005,0.0005,0.0006,0.0005,0.0006,0.0006,0.0005]
val_mae = [0.0671,0.0069,0.0020,0.0015,0.0014,0.0013,0.0010,0.0008,0.0008,0.0008,0.0007,0.0007,0.0007,0.0006,0.0006,0.0013,0.0004,0.0007,0.0006,0.0005,0.0005,0.0005,0.0007,0.0006,0.0007,0.0004,0.0006,0.0005,0.0004,0.0005]
epoca = []
for i in range(1,31):
    epoca.append(i)

In [ ]:
plt.plot(epoca, mae, 'g-',
         label='loss')
plt.plot(epoca, val_mae, 'm-',
         label='val_loss')
#plt.plot(epoca, val_mae,
#         'k-', label='Dados reais')
#plt.title('Loss', fontsize=15)
#plt.axis([0, 13, 0, 18000.0])
plt.xlabel('Epoca', fontsize=15)
plt.ylabel('MAE', fontsize=15) 
plt.legend()
plt.savefig('maesenoclassico.pdf')
plt.show()

In [ ]:
# Create naive prediction based on standardized data.
test_output = test_sales_std[MIN:]
naive_prediction = test_sales_std[MIN-1:-1]
mean_squared_error = np.mean(np.square(naive_prediction
                                       - test_output))
mean_abs_error = np.mean(np.abs(naive_prediction
                                - test_output))
print('naive test mse: ', mean_squared_error)
print('naive test mean abs: ', mean_abs_error)


In [ ]:
# Use trained model to predict the test data
inputs = torch.from_numpy(test_X)
inputs = inputs.to(device)
outputs = model(inputs)
predicted_test = outputs.cpu().detach().numpy()

# De-standardize output.
predicted_test = np.reshape(predicted_test,
                            (len(predicted_test)))
predicted_test = predicted_test * stddev + mean

# Plot test prediction.
x = range(len(test_sales)-MIN)
plt.plot(x, predicted_test, 'g-',
         label='Previsão')
plt.plot(x, test_sales[-(len(test_sales)-MIN):],
         'm-', label='Valores reais')
plt.title('Experimento clássico 1', fontsize = 14)
plt.axis([-1, 11, 2000, 16000.0])
plt.xlabel('Mês', fontsize = 14)
plt.ylabel('target', fontsize = 14)
plt.legend()
plt.savefig('comparacaoclassico.pdf')
plt.show()

In [ ]:
# Plotar a função original

x = np.linspace(0, 199, 199)

# plt.plot(x, y_predict, label='Função Original')

# Deslocar a função horizontalmente
def funcao_deslocada(y_predict, deslocamento):
    return y_predict + deslocamento

# Especificar o deslocamento desejado
deslocamento_horizontal = 500

# Plotar a função deslocada
plt.plot(train_sales0, 'k-', label='Treinamento')
#plt.plot(x+799, test_sales, 'k-', label='Treinamento')
plt.plot(x+800, predicted_test, 'g-', label=f'Previsão')

#plt.plot(x-500, funcao_deslocada(y_predict, deslocamento_horizontal), label=f'Função Deslocada ({deslocamento_horizontal} rad)')
novos_rotulos_x = [0, 200, 400, 600, 800, 1000]
plt.xticks(novos_rotulos_x)

# Adicionar legendas e título
plt.legend()
plt.title('f(x) = sen(x)', fontsize=14)
plt.xlabel('x', fontsize=14)
plt.ylabel('sen(x)', fontsize=14)
plt.savefig('seno-classico.pdf')
# Exibir o gráfico
plt.show()

In [ ]:
predicted_test

In [ ]:
test_sales[-(len(test_sales)-MIN):]